## 問題

https://twitter.com/e869120/status/1392612322410057729

## 解説

https://twitter.com/e869120/status/1392974101061378049

In [1]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [2]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [5]:
type Node = {
  Id: int
  Parent: int option
  Children: int list
  Depth: int
}

type Tree = {
  NodeDict: IReadOnlyDictionary<int, Node>
  RootId: int
  MaxDepth: int
} with
  member x.Count = x.NodeDict.Count

In [7]:
let getTree rootId edges =
  let nodeDict = edges |> Seq.collect (fun (x, y) -> [x, y; y, x]) |> Seq.groupBy fst |> Seq.map (fun (k, vs) -> k, vs |> Seq.map snd |> Seq.toList) |> dict
  let rec loop (result: Dictionary<int, Node>) depth = function
    | [] -> { NodeDict = result.AsReadOnly() :> IReadOnlyDictionary<_, _>; RootId = rootId; MaxDepth = depth - 1 }
    | nodeIdParents ->
      let next =
        ([], nodeIdParents) ||> fold (fun next (nodeId, parent) ->
          let children = nodeDict[nodeId] |> filter (fun n -> result.ContainsKey(n) |> not)
          result.Add(nodeId, { Id = nodeId; Parent = parent; Children = children; Depth = depth })
          (children |> map (fun c -> c, Some nodeId)) @ next)
      loop result (depth + 1) next
  loop (Dictionary()) 0 [rootId, None]

In [21]:
let rec dfs tree nodeId =
  tree.NodeDict[nodeId].Children |> Seq.sumBy (countDescendants tree) |> (+) 1
and countDescendants =
  Memoization.memoizeN dfs

In [22]:
let solve edges =
  let edges = edges |> Seq.toArray
  let tree = getTree 1 edges
  tree.NodeDict |> Seq.sumBy (fun (KeyValue(nodeId, node)) ->
    let descendantsCount = node.Parent |> Option.map (fun _ -> countDescendants tree nodeId) |> Option.defaultValue 0
    descendantsCount * (tree.Count - descendantsCount))

In [24]:
solve [
  1, 2
]

1

In [25]:
solve [
  1, 2
  1, 3
  1, 4
]

9

In [26]:
solve [
  1, 2
  3, 1
  4, 2
  2, 5
  3, 6
  3, 7
  8, 4
  4, 9
  10, 5
  11, 7
  7, 12
]

211